In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/forecasting-sticker-sales/sample_submission.csv
/kaggle/input/forecasting-sticker-sales/train.csv
/kaggle/input/forecasting-sticker-sales/test.csv


In [2]:
sticker_data = pd.read_csv('/kaggle/input/forecasting-sticker-sales/train.csv')
sticker_data.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,date,country,store,product,num_sold
0,0,2010-01-01,Canada,Discount Stickers,Holographic Goose,NaN
1,1,2010-01-01,Canada,Discount Stickers,Kaggle,973.0
2,2,2010-01-01,Canada,Discount Stickers,Kaggle Tiers,906.0
3,3,2010-01-01,Canada,Discount Stickers,Kerneler,423.0
4,4,2010-01-01,Canada,Discount Stickers,Kerneler Dark Mode,491.0


In [3]:
sticker_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230130 entries, 0 to 230129
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   id        230130 non-null  int64  
 1   date      230130 non-null  object 
 2   country   230130 non-null  object 
 3   store     230130 non-null  object 
 4   product   230130 non-null  object 
 5   num_sold  221259 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 10.5+ MB


# Because we are trying to predict the number sold, we cannot impute that data as it may interfere with our final results, so we'll drop all rows that have no num_sold data

In [4]:
sticker_data_cleaned = sticker_data[sticker_data['num_sold'].notna()]
sticker_data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 221259 entries, 1 to 230129
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   id        221259 non-null  int64  
 1   date      221259 non-null  object 
 2   country   221259 non-null  object 
 3   store     221259 non-null  object 
 4   product   221259 non-null  object 
 5   num_sold  221259 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 11.8+ MB


In [5]:
sticker_data['country'].unique()

array(['Canada', 'Finland', 'Italy', 'Kenya', 'Norway', 'Singapore'],
      dtype=object)

In [6]:
sticker_data['store'].unique()

array(['Discount Stickers', 'Stickers for Less', 'Premium Sticker Mart'],
      dtype=object)

In [7]:
sticker_data['product'].unique()

array(['Holographic Goose', 'Kaggle', 'Kaggle Tiers', 'Kerneler',
       'Kerneler Dark Mode'], dtype=object)

In [8]:
# Checking to see if there's any order among the different cards sold
sticker_data['product'].value_counts()

product
Holographic Goose     46026
Kaggle                46026
Kaggle Tiers          46026
Kerneler              46026
Kerneler Dark Mode    46026
Name: count, dtype: int64

Except for date, all features should be one-hot encoded and because we are predicting a continous variable with a large dataset ( > 100k), we will use the SGDRegressor.
We will convert date into an integer.

In [9]:
# See if our join was correct
df = sticker_data_cleaned.join(pd.get_dummies(sticker_data_cleaned['country'])).drop(columns='country')
df

,id,date,store,product,num_sold,Canada,Finland,Italy,Kenya,Norway,Singapore
1,1,2010-01-01,Discount Stickers,Kaggle,973.0,True,False,False,False,False,False
2,2,2010-01-01,Discount Stickers,Kaggle Tiers,906.0,True,False,False,False,False,False
3,3,2010-01-01,Discount Stickers,Kerneler,423.0,True,False,False,False,False,False
4,4,2010-01-01,Discount Stickers,Kerneler Dark Mode,491.0,True,False,False,False,False,False
5,5,2010-01-01,Stickers for Less,Holographic Goose,300.0,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
230125,230125,2016-12-31,Premium Sticker Mart,Holographic Goose,466.0,False,False,False,False,False,True
230126,230126,2016-12-31,Premium Sticker Mart,Kaggle,2907.0,False,False,False,False,False,True
230127,230127,2016-12-31,Premium Sticker Mart,Kaggle Tiers,2299.0,False,False,False,False,False,True
230128,230128,2016-12-31,Premium Sticker Mart,Kerneler,1242.0,False,False,False,False,False,True


In [10]:
sticker_data_cleaned = df
sticker_data_cleaned

,id,date,store,product,num_sold,Canada,Finland,Italy,Kenya,Norway,Singapore
1,1,2010-01-01,Discount Stickers,Kaggle,973.0,True,False,False,False,False,False
2,2,2010-01-01,Discount Stickers,Kaggle Tiers,906.0,True,False,False,False,False,False
3,3,2010-01-01,Discount Stickers,Kerneler,423.0,True,False,False,False,False,False
4,4,2010-01-01,Discount Stickers,Kerneler Dark Mode,491.0,True,False,False,False,False,False
5,5,2010-01-01,Stickers for Less,Holographic Goose,300.0,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
230125,230125,2016-12-31,Premium Sticker Mart,Holographic Goose,466.0,False,False,False,False,False,True
230126,230126,2016-12-31,Premium Sticker Mart,Kaggle,2907.0,False,False,False,False,False,True
230127,230127,2016-12-31,Premium Sticker Mart,Kaggle Tiers,2299.0,False,False,False,False,False,True
230128,230128,2016-12-31,Premium Sticker Mart,Kerneler,1242.0,False,False,False,False,False,True


In [11]:
df = sticker_data_cleaned.join(pd.get_dummies(sticker_data_cleaned['store'])).drop(columns='store')
df

,id,date,product,num_sold,Canada,Finland,Italy,Kenya,Norway,Singapore,Discount Stickers,Premium Sticker Mart,Stickers for Less
1,1,2010-01-01,Kaggle,973.0,True,False,False,False,False,False,True,False,False
2,2,2010-01-01,Kaggle Tiers,906.0,True,False,False,False,False,False,True,False,False
3,3,2010-01-01,Kerneler,423.0,True,False,False,False,False,False,True,False,False
4,4,2010-01-01,Kerneler Dark Mode,491.0,True,False,False,False,False,False,True,False,False
5,5,2010-01-01,Holographic Goose,300.0,True,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
230125,230125,2016-12-31,Holographic Goose,466.0,False,False,False,False,False,True,False,True,False
230126,230126,2016-12-31,Kaggle,2907.0,False,False,False,False,False,True,False,True,False
230127,230127,2016-12-31,Kaggle Tiers,2299.0,False,False,False,False,False,True,False,True,False
230128,230128,2016-12-31,Kerneler,1242.0,False,False,False,False,False,True,False,True,False


In [12]:
sticker_data_cleaned = df
sticker_data_cleaned

,id,date,product,num_sold,Canada,Finland,Italy,Kenya,Norway,Singapore,Discount Stickers,Premium Sticker Mart,Stickers for Less
1,1,2010-01-01,Kaggle,973.0,True,False,False,False,False,False,True,False,False
2,2,2010-01-01,Kaggle Tiers,906.0,True,False,False,False,False,False,True,False,False
3,3,2010-01-01,Kerneler,423.0,True,False,False,False,False,False,True,False,False
4,4,2010-01-01,Kerneler Dark Mode,491.0,True,False,False,False,False,False,True,False,False
5,5,2010-01-01,Holographic Goose,300.0,True,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
230125,230125,2016-12-31,Holographic Goose,466.0,False,False,False,False,False,True,False,True,False
230126,230126,2016-12-31,Kaggle,2907.0,False,False,False,False,False,True,False,True,False
230127,230127,2016-12-31,Kaggle Tiers,2299.0,False,False,False,False,False,True,False,True,False
230128,230128,2016-12-31,Kerneler,1242.0,False,False,False,False,False,True,False,True,False


In [13]:
df = sticker_data_cleaned.join(pd.get_dummies(sticker_data_cleaned['product'])).drop(columns='product')
df

,id,date,num_sold,Canada,Finland,Italy,Kenya,Norway,Singapore,Discount Stickers,Premium Sticker Mart,Stickers for Less,Holographic Goose,Kaggle,Kaggle Tiers,Kerneler,Kerneler Dark Mode
1,1,2010-01-01,973.0,True,False,False,False,False,False,True,False,False,False,True,False,False,False
2,2,2010-01-01,906.0,True,False,False,False,False,False,True,False,False,False,False,True,False,False
3,3,2010-01-01,423.0,True,False,False,False,False,False,True,False,False,False,False,False,True,False
4,4,2010-01-01,491.0,True,False,False,False,False,False,True,False,False,False,False,False,False,True
5,5,2010-01-01,300.0,True,False,False,False,False,False,False,False,True,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230125,230125,2016-12-31,466.0,False,False,False,False,False,True,False,True,False,True,False,False,False,False
230126,230126,2016-12-31,2907.0,False,False,False,False,False,True,False,True,False,False,True,False,False,False
230127,230127,2016-12-31,2299.0,False,False,False,False,False,True,False,True,False,False,False,True,False,False
230128,230128,2016-12-31,1242.0,False,False,False,False,False,True,False,True,False,False,False,False,True,False


In [14]:
sticker_data_cleaned = df
sticker_data_cleaned

,id,date,num_sold,Canada,Finland,Italy,Kenya,Norway,Singapore,Discount Stickers,Premium Sticker Mart,Stickers for Less,Holographic Goose,Kaggle,Kaggle Tiers,Kerneler,Kerneler Dark Mode
1,1,2010-01-01,973.0,True,False,False,False,False,False,True,False,False,False,True,False,False,False
2,2,2010-01-01,906.0,True,False,False,False,False,False,True,False,False,False,False,True,False,False
3,3,2010-01-01,423.0,True,False,False,False,False,False,True,False,False,False,False,False,True,False
4,4,2010-01-01,491.0,True,False,False,False,False,False,True,False,False,False,False,False,False,True
5,5,2010-01-01,300.0,True,False,False,False,False,False,False,False,True,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230125,230125,2016-12-31,466.0,False,False,False,False,False,True,False,True,False,True,False,False,False,False
230126,230126,2016-12-31,2907.0,False,False,False,False,False,True,False,True,False,False,True,False,False,False
230127,230127,2016-12-31,2299.0,False,False,False,False,False,True,False,True,False,False,False,True,False,False
230128,230128,2016-12-31,1242.0,False,False,False,False,False,True,False,True,False,False,False,False,True,False


In [15]:
# Divide by 10^11 to get rid of unnessecary zeros
pd.to_datetime(sticker_data_cleaned['date']).astype(int) // 10 ** 11

1         12623040
2         12623040
3         12623040
4         12623040
5         12623040
            ...   
230125    14831424
230126    14831424
230127    14831424
230128    14831424
230129    14831424
Name: date, Length: 221259, dtype: int64

In [16]:
sticker_data_cleaned['date'] = pd.to_datetime(sticker_data_cleaned['date']).astype(int) // 10 ** 11
sticker_data_cleaned

,id,date,num_sold,Canada,Finland,Italy,Kenya,Norway,Singapore,Discount Stickers,Premium Sticker Mart,Stickers for Less,Holographic Goose,Kaggle,Kaggle Tiers,Kerneler,Kerneler Dark Mode
1,1,12623040,973.0,True,False,False,False,False,False,True,False,False,False,True,False,False,False
2,2,12623040,906.0,True,False,False,False,False,False,True,False,False,False,False,True,False,False
3,3,12623040,423.0,True,False,False,False,False,False,True,False,False,False,False,False,True,False
4,4,12623040,491.0,True,False,False,False,False,False,True,False,False,False,False,False,False,True
5,5,12623040,300.0,True,False,False,False,False,False,False,False,True,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230125,230125,14831424,466.0,False,False,False,False,False,True,False,True,False,True,False,False,False,False
230126,230126,14831424,2907.0,False,False,False,False,False,True,False,True,False,False,True,False,False,False
230127,230127,14831424,2299.0,False,False,False,False,False,True,False,True,False,False,False,True,False,False
230128,230128,14831424,1242.0,False,False,False,False,False,True,False,True,False,False,False,False,True,False
